In [8]:
import ccxt
import pandas as pd
from datetime import datetime, timedelta
import psycopg2

In [9]:
#config
conn = psycopg2.connect(database = "symbole", 
                        user = "postgres", 
                        host= 'localhost',
                        password = "postgresql_test1234567890Password",
                        port = 5432)
cursor = conn.cursor()


In [10]:

symbols = [
    'BTC/FDUSD',
    'BTC/USDT',
    'LINK/USDT'
]

In [11]:
binance = ccxt.binance()

def get_historical_data(symbol, timeframe='1d', limit=1000, start_time=None):
    all_data = []
    if start_time is None:
        start_time = binance.parse8601('2024-01-01T01:51:00Z')

    while True:
        ohlcv = binance.fetch_ohlcv(symbol, timeframe=timeframe, since=start_time, limit=limit)
        if len(ohlcv) == 0:
            break
        all_data += ohlcv
        start_time = ohlcv[-1][0] + 1
        print(f"Récupéré {len(all_data)} bougies pour {symbol}")

    data = pd.DataFrame(all_data, columns=['DateTime', 'Open', 'High', 'Low', 'Close', 'Volume'])
    data['DateTime'] = pd.to_datetime(data['DateTime'], unit='ms')
    return data

def create_table(symbol):
    table_name = symbol.replace("/", "_").lower()
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        datetime TIMESTAMP PRIMARY KEY,
        open NUMERIC,
        high NUMERIC,
        low NUMERIC,
        close NUMERIC,
        volume NUMERIC
    );
    """
    cursor.execute(create_table_query)
    conn.commit()

def insert_data_to_postgresql(symbol, data):
    table_name = symbol.replace("/", "_").lower()
    for _, row in data.iterrows():
        insert_query = f"""
        INSERT INTO {table_name} (datetime, open, high, low, close, volume)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (datetime) DO NOTHING;
        """
        cursor.execute(insert_query, (row['DateTime'], row['Open'], row['High'], row['Low'], row['Close'], row['Volume']))
    conn.commit()




In [12]:
for symbol in symbols:
    print(f"Traitement des données pour {symbol}...")
    data = get_historical_data(symbol, limit=1000)
    create_table(symbol)
    insert_data_to_postgresql(symbol, data)
    print(f"Données insérées pour {symbol}")

cursor.close()
conn.close()

Traitement des données pour BTC/FDUSD...
Récupéré 386 bougies pour BTC/FDUSD
Données insérées pour BTC/FDUSD
Traitement des données pour BTC/USDT...
Récupéré 386 bougies pour BTC/USDT
Données insérées pour BTC/USDT
Traitement des données pour LINK/USDT...
Récupéré 386 bougies pour LINK/USDT
Données insérées pour LINK/USDT
